In [ ]:
# Load protected areas dataset
protected_areas = ee.FeatureCollection("WCMC/WDPA/202106/polygons")

# Define filters
marine_filter = ee.Filter.eq("MARINE", "0")
not_mpa_filter = ee.Filter.neq("DESIG_ENG", "Marine Protected Area")
status_filter = ee.Filter.inList("STATUS", ["Designated", "Established", "Inscribed"])
designation_filter = ee.Filter.neq("DESIG_ENG", "UNESCO-MAB Biosphere Reserve")
excluded_pids = ["555655917", "555656005", "555656013", "555665477", "555656021",
                 "555665485", "555556142", "187", "555703455", "555563456", "15894"]
area_filter = ee.Filter.gte("GIS_AREA", 200)

# Filter to exclude specific WDPA_PID values
pids_filter = ee.Filter.inList("WDPA_PID", excluded_pids).Not()

# Combine all filters
combined_filter = ee.Filter.And(
    marine_filter,
    not_mpa_filter,
    status_filter,
    designation_filter,
    pids_filter,
    area_filter
)

# Apply filters
data = protected_areas.filter(combined_filter)

# Get WDPA_PIDs
wdpa_pids = data.aggregate_array("WDPA_PID").getInfo()
print(f"Number of protected areas: {len(wdpa_pids)}")